In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

!sudo apt update
!sudo apt install python3-pip
!sudo apt install python-pip
#!pip install open3d

In [ ]:
cd drive/MyDrive/Colab/pvcnn

In [ ]:
!pip install -r requirements.txt
!sudo apt-get install libsparsehash-dev
#!pip install --upgrade git+https://github.com/mit-han-lab/torchsparse.git@v1.4.0
#!pip install -U kaleido
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force
!sudo apt-get install xvfb

In [ ]:
 # data preprocessing without any label # Finished!

!python data/s3dis/prepare_data.py -d ./datasets/S3DIS/

In [5]:
# data evaluation without any label # Finished!
# ~~~.prediction(i).npy files contain i'th room's segmentation result

!python train.py configs/s3dis/pvcnn2/area5/c1.py --devices 0 --evaluate --configs.evaluate.best_checkpoint_path s3dis.pvcnn2.area5.c1.pth.tar

==> loading configs from ['configs/s3dis/pvcnn2/area5/c1.py']
[seed] = 1588147245
[deterministic] = False
[data]
  [num_workers] = 16
  [num_classes] = 13
[dataset]
  [func] = <class 'datasets.s3dis.S3DIS'>
  [root] = data/s3dis/pointcnn
  [with_normalized_coords] = True
  [num_points] = 8192
  [holdout_area] = 5
  [split] = test
[evaluate]
  [fn] = <function evaluate at 0x7fd5a2201560>
  [num_votes] = 1
  [batch_size] = 1
  [dataset]
    [split] = test
  [best_checkpoint_path] = s3dis.pvcnn2.area5.c1.pth.tar
  [predictions_path] = s3dis.pvcnn2.area5.c1.predictions
  [stats_path] = s3dis.pvcnn2.area5.c1.eval.npy
[train]
  [num_epochs] = 50
  [batch_size] = 32
  [meters]
    [acc/iou_{}]
      [func] = <class 'meters.s3dis.MeterS3DIS'>
      [metric] = iou
      [num_classes] = 13
    [acc/acc_{}]
      [func] = <class 'meters.s3dis.MeterS3DIS'>
      [metric] = overall
      [num_classes] = 13
  [metric] = acc/iou_test
  [criterion]
    [func] = <class 'torch.nn.modules.loss.CrossEntro

In [ ]:
# common visualization with original data
import numpy as np
import open3d as o3d

origin = "./office_1.txt"
output_path = "./"

point_cloud = np.loadtxt(origin)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud[:,:3])
pcd.colors = o3d.utility.Vector3dVector(point_cloud[:,3:6]/255)
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
o3d.visualization.draw_geometries([pcd])


In [ ]:
# point cloud segmentation result visualization # Finished!
import numpy as np
import open3d as o3d

segmentation = np.load("./predictions.npy", allow_pickle = True)
segmentation_backup = np.load("./prediction_backup.npy", allow_pickle = True)
COLOR_MAP = np.array([[255,  0,  0], [ 30, 30,255], [ 60,179,113], [238,130,238], [255,165,  0],
                      [106, 90,205], [ 60, 60, 60], [180,130,130], [200,140,240], [107, 99, 71],
                      [191,179, 71], [191, 25,221], [ 53,199,221], [  0,100,136], [102,  0,102],
                      [ 51,  0,153], [153,102,204], [204,153,  0], [255,204,153], [  0,204,204],
                      [  0, 51, 51], [153,  0,  0], [102,153,255], [204,255,  0], [153, 51,204],
                      [204,102,  0], [255,153,102], [ 51,204,102], [ 51,153,102], [255,255,255]], dtype=float)

map_result = COLOR_MAP[segmentation].tolist()
for i in range(0, len(map_result)):
  map_result[i][0] /= 255.0
  map_result[i][1] /= 255.0
  map_result[i][2] /= 255.0

point_cloud = np.loadtxt("./office_1.txt")
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud[:,:3])
pcd.colors = o3d.utility.Vector3dVector(map_result)
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
o3d.visualization.draw_geometries([pcd])

In [ ]:
# leaving only one label
# 8: table, 9: chair, 11: bookcase, etc..
import numpy as np
import open3d as o3d

point_cloud = np.loadtxt("./office_1.txt")
segmentation = np.load("./predictions.npy", allow_pickle = True)
target_label = 12

saved_cloud = []
for i in range(0, len(point_cloud)):
  if(segmentation[i] == target_label):
    saved_cloud.append(point_cloud[i, :].tolist())

result = np.array(saved_cloud)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(result[:,:3])
pcd.colors = o3d.utility.Vector3dVector(result[:,3:6]/255)
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
o3d.visualization.draw_geometries([pcd])


In [ ]:
# object level clustering
import numpy as np
import open3d as o3d

point_cloud = np.loadtxt("./office_1.txt")
segmentation = np.load("./predictions.npy", allow_pickle = True)
target_label = 9

saved_cloud = []
for i in range(0, len(point_cloud)):
  if(segmentation[i] == target_label):
    saved_cloud.append(point_cloud[i, :].tolist())

result = np.array(saved_cloud)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(result[:,:3])

with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(
        pcd.cluster_dbscan(eps=0.08, min_points=50, print_progress=True))

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")

COLOR_MAP = np.array([[255,  0,  0], [ 30, 30,255], [ 60,179,113], [238,130,238], [255,165,  0],
                      [106, 90,205], [ 60, 60, 60], [180,130,130], [200,140,240], [107, 99, 71],
                      [191,179, 71], [191, 25,221], [ 53,199,221], [  0,100,136], [102,  0,102],
                      [ 51,  0,153], [153,102,204], [204,153,  0], [255,204,153], [  0,204,204],
                      [  0, 51, 51], [153,  0,  0], [102,153,255], [204,255,  0], [153, 51,204],
                      [204,102,  0], [255,153,102], [ 51,204,102], [ 51,153,102], [255,255,255]], dtype=float)

map_result = COLOR_MAP[labels % 30].tolist()
for i in range(0, len(map_result)):
  map_result[i][0] /= 255.0
  map_result[i][1] /= 255.0
  map_result[i][2] /= 255.0


pcd.colors = o3d.utility.Vector3dVector(map_result)
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
o3d.visualization.draw_geometries([pcd])
o3d.io.write_point_cloud("./chair.pcd", pcd)


In [ ]:
# object level clustering with voxel downsampling
import numpy as np
import open3d as o3d

point_cloud = np.loadtxt("./office_1.txt")
segmentation = np.load("./predictions.npy", allow_pickle = True)
target_label = 9

saved_cloud = []
for i in range(0, len(point_cloud)):
  if(segmentation[i] == target_label):
    saved_cloud.append(point_cloud[i, :].tolist())

result = np.array(saved_cloud)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(result[:,:3])
downpcd = pcd.voxel_down_sample(voxel_size=0.05)

with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(
        downpcd.cluster_dbscan(eps=0.3, min_points=50, print_progress=True))

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")

COLOR_MAP = np.array([[255,  0,  0], [ 30, 30,255], [ 60,179,113], [238,130,238], [255,165,  0],
                      [106, 90,205], [ 60, 60, 60], [180,130,130], [200,140,240], [107, 99, 71],
                      [191,179, 71], [191, 25,221], [ 53,199,221], [  0,100,136], [102,  0,102],
                      [ 51,  0,153], [153,102,204], [204,153,  0], [255,204,153], [  0,204,204],
                      [  0, 51, 51], [153,  0,  0], [102,153,255], [204,255,  0], [153, 51,204],
                      [204,102,  0], [255,153,102], [ 51,204,102], [ 51,153,102], [255,255,255]])

map_result = COLOR_MAP[labels % 30].tolist()
for i in range(0, len(map_result)):
  map_result[i][0] /= 255.0
  map_result[i][1] /= 255.0
  map_result[i][2] /= 255.0


downpcd.colors = o3d.utility.Vector3dVector(map_result)
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
o3d.visualization.draw_geometries([downpcd])
o3d.io.write_point_cloud("./chair_voxel.pcd", downpcd)

In [ ]:
# object extraction from clustered objects with only 2000 points over.
import numpy as np
import open3d as o3d

AT_LEAST = 2000
pcd = o3d.io.read_point_cloud("./chair.pcd")
COLOR_MAP = np.array([[255,  0,  0], [ 30, 30,255], [ 60,179,113], [238,130,238], [255,165,  0],
                      [106, 90,205], [ 60, 60, 60], [180,130,130], [200,140,240], [107, 99, 71],
                      [191,179, 71], [191, 25,221], [ 53,199,221], [  0,100,136], [102,  0,102],
                      [ 51,  0,153], [153,102,204], [204,153,  0], [255,204,153], [  0,204,204],
                      [  0, 51, 51], [153,  0,  0], [102,153,255], [204,255,  0], [153, 51,204],
                      [204,102,  0], [255,153,102], [ 51,204,102], [ 51,153,102], [255,255,255]], dtype=float)

for x in range (0, len(COLOR_MAP) - 2):
  target = COLOR_MAP[x]
  target[0] /= 255.0
  target[1] /= 255.0
  target[2] /= 255.0
  print(target[0])
  saved_points = []
  saved_colors = []
  colors = np.asarray(pcd.colors)
  points = np.asarray(pcd.points)
  for i in range(0, len(pcd.colors)):
    if (target[0] == pcd.colors[i][0] and target[1] == pcd.colors[i][1] and target[2] == pcd.colors[i][2]):
      saved_points.append(points[i, :].tolist())
      saved_colors.append(colors[i, :].tolist())
  
  result_points = np.array(saved_points)
  result_colors = np.array(saved_colors)
  if(len(result_points) > AT_LEAST):
    pcd2 = o3d.geometry.PointCloud()
    pcd2.points = o3d.utility.Vector3dVector(result_points[:,:])
    pcd2.colors = o3d.utility.Vector3dVector(result_colors[:,:])
    pcd2.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    o3d.visualization.draw_geometries([pcd2])



In [ ]:
# fetch.py
# make pcd files for every label
import numpy as np
import open3d as o3d
import os

point_cloud = np.loadtxt("./office_1.txt")
segmentation = np.load("./predictions.npy", allow_pickle = True)
NUM_LABEL = segmentation.max()
if not os.path.exists('fetched'):
  os.makedirs('fetched')

for TARGET in range (0, NUM_LABEL+1):
  saved_cloud = []
  for i in range(0, len(point_cloud)):
    if(segmentation[i] == TARGET):
      saved_cloud.append(point_cloud[i, :].tolist())
  
  if len(saved_cloud) == 0:
    continue
  
  result = np.array(saved_cloud)
  pcd = o3d.geometry.PointCloud()
  pcd.points = o3d.utility.Vector3dVector(result[:,:3])
  with o3d.utility.VerbosityContextManager(
          o3d.utility.VerbosityLevel.Debug) as cm:
      labels = np.array(
          pcd.cluster_dbscan(eps=0.08, min_points=50, print_progress=True))
  
  max_label = labels.max()
  print(f"point cloud has {max_label + 1} clusters")
  COLOR_MAP = np.array([[255,  0,  0], [ 30, 30,255], [ 60,179,113], [238,130,238], [255,165,  0],
                        [106, 90,205], [ 60, 60, 60], [180,130,130], [200,140,240], [107, 99, 71],
                        [191,179, 71], [191, 25,221], [ 53,199,221], [  0,100,136], [102,  0,102],
                        [ 51,  0,153], [153,102,204], [204,153,  0], [255,204,153], [  0,204,204],
                        [  0, 51, 51], [153,  0,  0], [102,153,255], [204,255,  0], [153, 51,204],
                        [204,102,  0], [255,153,102], [ 51,204,102], [ 51,153,102], [255,255,255]], dtype=float)
  map_result = COLOR_MAP[labels % 30].tolist()
  for i in range(0, len(map_result)):
    map_result[i][0] /= 255.0
    map_result[i][1] /= 255.0
    map_result[i][2] /= 255.0
  
  pcd.colors = o3d.utility.Vector3dVector(map_result)
  pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
  #o3d.visualization.draw_geometries([pcd])
  o3d.io.write_point_cloud("./fetched/" + str(TARGET) + ".pcd", pcd)

In [ ]:
#extract.py

import numpy as np
import open3d as o3d
import os
from os import listdir
from os.path import isfile, join

AT_LEAST = 2000
onlyfiles = [f for f in listdir("fetched") if isfile(join("fetched", f))]
for CURRENT in onlyfiles:
    pcd = o3d.io.read_point_cloud(join("fetched", CURRENT))
    current_directory = join("fetched", CURRENT.split('.')[0])
    if not os.path.exists(current_directory):
        os.makedirs(current_directory)
    
    print("FILE: " + CURRENT)
    NUM_OBJECTS = 0
    COLOR_MAP = np.array([[255,  0,  0], [ 30, 30,255], [ 60,179,113], [238,130,238], [255,165,  0],
                        [106, 90,205], [ 60, 60, 60], [180,130,130], [200,140,240], [107, 99, 71],
                        [191,179, 71], [191, 25,221], [ 53,199,221], [  0,100,136], [102,  0,102],
                        [ 51,  0,153], [153,102,204], [204,153,  0], [255,204,153], [  0,204,204],
                        [  0, 51, 51], [153,  0,  0], [102,153,255], [204,255,  0], [153, 51,204],
                        [204,102,  0], [255,153,102], [ 51,204,102], [ 51,153,102], [255,255,255]], dtype=float)
                        
    for x in range (0, len(COLOR_MAP) - 2):
        target = COLOR_MAP[x]
        print("    COLOR: " + str(target))
        target[0] /= 255.0
        target[1] /= 255.0
        target[2] /= 255.0

        saved_points = []
        saved_colors = []
        colors = np.asarray(pcd.colors)
        points = np.asarray(pcd.points)
        for i in range(0, len(pcd.colors)):
            if (target[0] == pcd.colors[i][0] and target[1] == pcd.colors[i][1] and target[2] == pcd.colors[i][2]):
                saved_points.append(points[i, :].tolist())
                saved_colors.append(colors[i, :].tolist())
        
        result_points = np.array(saved_points)
        result_colors = np.array(saved_colors)
        if(len(result_points) > AT_LEAST):
            print("                            NUM_OBJECTS: " + str(NUM_OBJECTS))
            pcd2 = o3d.geometry.PointCloud()
            pcd2.points = o3d.utility.Vector3dVector(result_points[:,:])
            pcd2.colors = o3d.utility.Vector3dVector(result_colors[:,:])
            pcd2.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
            #o3d.visualization.draw_geometries([pcd2])
            o3d.io.write_point_cloud(join(current_directory, str(NUM_OBJECTS) + ".pcd"), pcd)
            NUM_OBJECTS = NUM_OBJECTS +1
